In [ ]:
#Load libraries
library(plyr)
library(tidyverse)
library(pander)
library(ggpubr)
library(rapportools)
library(ggplot2)
#Get working directory
getwd()
#Set working directory
setwd('/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment')

In [7]:
cd /gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/hearing_impairment
head -n1 200804_UKBB_HI_genotypeqc.csv > header.txt
cat 200713_UKBB_excluded_individuals.csv >> header.txt

In [ ]:
cat 200713_UKBB_excluded_individuals.csv | head 

In [3]:
df_excluded <- read.csv(file = '200713_UKBB_excluded_individuals_new.csv', header=TRUE)
dim(df_excluded)

[1] 12397  1074

In [4]:
head(df_excluded)

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.34.0.0,f.53.0.0,f.53.1.0,⋯,f.41281.0.37,f.41281.0.38,f.41281.0.39,f.41281.0.40,f.41281.0.41,f.41281.0.42,f.41281.0.43,f.41281.0.44,f.41281.0.45,f.41281.0.46
,<int>,<int>,<int>,<int>,<int>,<int>,<lgl>,<int>,<fct>,<fct>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,1000947,1000947,0,0,2,-9,NA,1941,2008-06-04,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1002216,1002216,0,0,1,-9,NA,1945,2008-05-16,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,1002460,1002460,0,0,1,-9,NA,1947,2010-05-05,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,1002510,1002510,0,0,1,-9,NA,1956,2009-02-04,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,1002569,1002569,0,0,2,-9,NA,1949,2009-10-24,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,1002587,1002587,0,0,1,-9,NA,1954,2009-03-16,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [5]:
table(df_excluded$f.3393.0.0)


         Do not know           Moderately           Not at all 
                   9                  423                  498 
Prefer not to answer             Severely             Slightly 
                   1                  126                  794 

In [6]:
## Obtain the number of individuals with f.2247

# Recode function:
recode<-function(df,column_name){
  new_names<-c()
  for (i in column_name){
    new_column_name<-paste0(i,"_recode")
    new_names<-c(new_names,new_column_name)
    df[,new_column_name] <- revalue(df[,i], c("No"= 0, 
                                            "Yes" =1,
                                            "Prefer not to answer"= NA,
                                             "Do not know"=9,
                                             "I am completely deaf"=NA))
  }
  return (list(df=df,new_column_names=new_names))
}

# columns needs to be recoded:
column_name<-c("f.2247.0.0","f.2247.1.0","f.2247.2.0","f.2247.3.0")

# get a new data.frame with recoded columns added:
df_excluded<-recode(df=df_excluded,column_name)$df

# get names of recoded columns:
new_column_names<-recode(df=df_excluded,column_name)$new_column_names

# show recode summary:
for (i in new_column_names)
{cat(i,"summary:");print(table(df_excluded[,i]));cat("\n")}

The following `from` values were not present in `x`: I am completely deaf

The following `from` values were not present in `x`: Prefer not to answer, I am completely deaf

The following `from` values were not present in `x`: Prefer not to answer, I am completely deaf

The following `from` values were not present in `x`: Prefer not to answer, I am completely deaf

The following `from` values were not present in `x`: I am completely deaf

The following `from` values were not present in `x`: Prefer not to answer, I am completely deaf

The following `from` values were not present in `x`: Prefer not to answer, I am completely deaf

The following `from` values were not present in `x`: Prefer not to answer, I am completely deaf



f.2247.0.0_recode summary:
   9    0    1 
 190 5594 6576 

f.2247.1.0_recode summary:
  9   0   1 
 21 286 358 

f.2247.2.0_recode summary:
  9   0   1 
 22 498 656 

f.2247.3.0_recode summary:
 9  0  1 
 2 33 57 



In [7]:
# Extract subset of data only with the recode columns of tinnitus
data_sub <- df_excluded %>%
  select("IID",all_of(new_column_names)) 

# Function to extract all the available answers for 4 visits
# and put them in one string as "0000", "111", "991", etc

f2<-function(x){
  visit<-''
  for (i in 2:5){ # do not take the first column (i.e. IID)
    if (!is.na(x[i]))
    {visit<-paste0(visit,x[i])}
  }
  if(is.null(visit)){visit=NA}
  else{visit=visit}
  return (visit)
}

# Apply the above function and remove NAs
data_sub$visit<-apply(data_sub, 1, f2)

names(table(data_sub$visit))

[1] ""     "0"    "00"   "000"  "0000" "001"  "0011" "009"  "01"   "010" 
[11] "0100" "011"  "0111" "09"   "090"  "099"  "1"    "10"   "100"  "101" 
[21] "109"  "1091" "11"   "110"  "1101" "111"  "1111" "19"   "191"  "1990"
[31] "9"    "90"   "91"   "911"  "99"

In [8]:
table(data_sub$visit)
all_pattern<-unique(data_sub$visit)


        0   00  000 0000  001 0011  009   01  010 0100  011 0111   09  090  099 
  36 4823  423   95    8   13    2    3  164   10    1   29    1   20    1    1 
   1   10  100  101  109 1091   11  110 1101  111 1111   19  191 1990    9   90 
5792   93   15   10    1    1  520    5    1  119    7   10    2    1  166    7 
  91  911   99 
  10    4    3 

In [9]:
all_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("9",all_pattern) & !grepl("1",all_pattern))]
all_yes<-all_pattern[which(!grepl("0",all_pattern) & !grepl("9",all_pattern) & grepl("1",all_pattern))]
all_do_not_know<-all_pattern[which(!grepl("0",all_pattern) & grepl("9",all_pattern) & !grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%union(union(all_no,all_yes),all_do_not_know),"visit"])


   0   00  000 0000    1   11  111 1111    9   99 
4823  423   95    8 5792  520  119    7  166    3 

In [10]:
consistent_1<-c(all_no,all_yes,all_do_not_know)

# might be inconsistent cases:
do_not_know_no<-all_pattern[which(grepl("0",all_pattern) & grepl("9",all_pattern) & !grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%do_not_know_no,"visit"])


009  09 090 099  90 
  3  20   1   1   7 

In [11]:
do_not_know_yes<-all_pattern[which(!grepl("0",all_pattern) & grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%do_not_know_yes,"visit"])


 19 191  91 911 
 10   2  10   4 

In [12]:
yes_no<-all_pattern[which(grepl("0",all_pattern) & !grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%yes_no,"visit"])


 001 0011   01  010 0100  011 0111   10  100  101  110 1101 
  13    2  164   10    1   29    1   93   15   10    5    1 

In [13]:
with_all_three<-all_pattern[which(grepl("0",all_pattern) & grepl("9",all_pattern) & grepl("1",all_pattern))]
table(data_sub[data_sub$visit%in%with_all_three,"visit"])


 109 1091 1990 
   1    1    1 

In [14]:
might_inconsistent<-c(do_not_know_no,do_not_know_yes,yes_no,with_all_three)

In [17]:
# get all the inconsistent cases:
exceptions<-c("91","911","9111","991","0001","001","0011","01","011","0111", "0091", "091","0911","0991","9001","901","9011")
inconsistent<-setdiff(might_inconsistent,exceptions)

cat("There are",length(which(is.empty(data_sub$visit))),"NAs, removed")

There are 36 NAs, removed

In [18]:
cat("There are",length(which(data_sub$visit%in%inconsistent)),"inconsistent cases, removed")

There are 182 inconsistent cases, removed

In [19]:
# removed those NAs, consistent for do not know - 9 and inconsistent cases: 
IID_hearing_diff<-data_sub %>%
  filter(!is.empty(visit)) %>%
  filter((!visit%in%inconsistent)) %>%
  filter((!visit%in%all_do_not_know)) %>%
  select(IID)

dim(IID_hearing_diff)

[1] 12010     1

In [20]:
IID_hearing_diff<-IID_hearing_diff[,1]
cat("After removing all the NAs and inconsistent cases, there are",length(IID_hearing_diff),"cases left")

After removing all the NAs and inconsistent cases, there are 12010 cases left

In [22]:
# get the dataset after cleansing: 
df.hearing.diff<-df_excluded %>%
  filter(IID%in%IID_hearing_diff)

dim(df.hearing.diff)

[1] 12010  1078

In [23]:
hearing_diff <- df.hearing.diff %>% 
  select(IID,FID,f.22001.0.0, f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode)
head(hearing_diff)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,1000947,1000947,NA,NA,NA,NA,NA,0,NA,NA,NA
2,1002216,1002216,NA,NA,NA,NA,NA,1,NA,NA,NA
3,1002460,1002460,NA,NA,NA,NA,NA,1,NA,NA,NA
4,1002510,1002510,NA,NA,NA,NA,NA,0,NA,NA,NA
5,1002569,1002569,NA,NA,NA,NA,NA,1,NA,NA,NA
6,1002587,1002587,NA,NA,NA,NA,NA,0,NA,0,NA


In [25]:
data_sub <- select(hearing_diff,starts_with("f.2247"))

# Function to define cases
f<-function(x){
  visit<-c()
  for (i in 1:4){
    if (!is.na(x[i]))
    {visit<-c(visit,x[i])}
  }
  if ("1" %in% visit){result= TRUE}
  else{result=FALSE}
  return (result)
}

# Apply the above function
hearing_diff$cases<-apply(data_sub, 1, f)
head(hearing_diff,10)

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000947,1000947,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
2,1002216,1002216,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
3,1002460,1002460,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
4,1002510,1002510,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
5,1002569,1002569,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
6,1002587,1002587,NA,NA,NA,NA,NA,0,NA,0,NA,FALSE
7,1002621,1002621,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
8,1003440,1003440,S018,M5459,N390,R13,R15,1,NA,NA,NA,TRUE
9,1003529,1003529,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE


In [26]:
hearing_diff_cases <- hearing_diff %>% 
  filter(cases == TRUE)
head(hearing_diff_cases)
cat(dim(hearing_diff_cases)[1], "individuals are cases for f.2247")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1002216,1002216,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
2,1002460,1002460,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
3,1002569,1002569,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
4,1003440,1003440,S018,M5459,N390,R13,R15,1,NA,NA,NA,TRUE
5,1004303,1004303,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE
6,1004509,1004509,NA,NA,NA,NA,NA,1,NA,NA,NA,TRUE


6661 individuals are cases for f.2247

In [44]:
hearing_diff_controls <- hearing_diff %>% 
  filter(cases == FALSE)
head(hearing_diff_controls)
cat(dim(hearing_diff_controls)[1], "individuals are controls for f.2247")

,IID,FID,f.22001.0.0,f.21003.0.0,f.21003.1.0,f.21003.2.0,f.21003.3.0,f.2247.0.0_recode,f.2247.1.0_recode,f.2247.2.0_recode,f.2247.3.0_recode,cases
,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<lgl>
1,1000947,1000947,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
2,1002510,1002510,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
3,1002587,1002587,NA,NA,NA,NA,NA,0,NA,0,NA,FALSE
4,1002621,1002621,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
5,1003529,1003529,NA,NA,NA,NA,NA,0,NA,NA,NA,FALSE
6,1004098,1004098,NA,S3270,NA,NA,NA,0,NA,NA,NA,FALSE


5349 individuals are controls for f.2247

In [28]:
any(apply(df_excluded, 2, function(x) grepl("H809", x)))

[1] TRUE

In [ ]:
H809 <- grep(df_excluded,("H809"))

In [ ]:
table(H809)